### Solutions

#### Question 1

Write a decorator that can be used to print out how long a function takes to run.

In [1]:
from functools import wraps
from time import perf_counter

def timed(fn):
    '''Decorator that prints how long the wrapped function takes to run.'''
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        print(f'{fn.__name__} took {elapsed:.6f} seconds')
        return result
    return wrapper

# Example usage
@timed
def slow_add(x, y):
    from time import sleep
    sleep(1)
    return x + y


#### Question 2

Write a decorator named `normalize` that can be used to decorate numeric functions to ensure that the result is always returned as a `float` with a precision defined by some global variable `PRECISION`.

In [2]:
from decimal import Decimal
from functools import wraps

PRECISION = 2  # can be changed at runtime

def normalize(fn):
    '''Decorator that ensures the wrapped function returns a float
    rounded to the current PRECISION value.'''
    @wraps(fn)
    def wrapper(*args, **kwargs):
        raw_result = fn(*args, **kwargs)
        try:
            return round(float(raw_result), PRECISION)
        except (TypeError, ValueError) as exc:
            raise TypeError(
                f'Function {fn.__name__} did not return a numeric result: {raw_result!r}'
            ) from exc
    return wrapper

@normalize
def perc_diff(x, y):
    try:
        return (y - x) / x * 100
    except ZeroDivisionError:
        return 0

@normalize
def sum_squares(*args):
    return sum(x ** 2 for x in args)

@normalize
def avg(*args):
    if len(args) == 0:
        return 0
    numbers = [Decimal(x) for x in args]
    total = sum(numbers)
    return total / len(numbers)


#### Question 3

Use Python's LRU caching decorator to help improve performance when this function is called multiple times with the same arguments, and then use `timeit` to test how performance is affected.

In [3]:
from functools import lru_cache
from time import sleep
import timeit

@lru_cache(maxsize=None)
def add(x, y):
    '''Simulates a long-running deterministic function.'''
    sleep(2)
    return x + y

def measure_no_cache():
    '''Call the underlying function (no cache) three times
    with the same arguments.'''
    for _ in range(3):
        add.__wrapped__(10, 20)  # bypass cache

def measure_with_cache():
    '''Call the cached function three times
    with the same arguments.'''
    for _ in range(3):
        add(10, 20)

# Time without cache
add.cache_clear()
t_no_cache = timeit.timeit(measure_no_cache, number=1)

# Time with cache
add.cache_clear()
t_with_cache = timeit.timeit(measure_with_cache, number=1)

print(f'Time without cache: {t_no_cache:.4f} seconds')
print(f'Time with cache:    {t_with_cache:.4f} seconds')


Time without cache: 6.0023 seconds
Time with cache:    2.0005 seconds


#### Question 4

Implement a decorator factory `normalize(precision)` so we can use different precisions per function: `@normalize(2)`, `@normalize(4)`, etc.

In [4]:
from functools import wraps
from decimal import Decimal

def normalize(precision):
    '''Decorator factory that rounds the function result to the given
    number of decimal places and returns a float.'''
    def decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            raw_result = fn(*args, **kwargs)
            try:
                return round(float(raw_result), precision)
            except (TypeError, ValueError) as exc:
                raise TypeError(
                    f'Function {fn.__name__} did not return a numeric result: {raw_result!r}'
                ) from exc
        return wrapper
    return decorator

@normalize(2)
def perc_diff(x, y):
    try:
        return (y - x) / x * 100
    except ZeroDivisionError:
        return 0

@normalize(4)
def sum_squares(*args):
    return sum(x ** 2 for x in args)

@normalize(8)
def avg(*args):
    if len(args) == 0:
        return 0
    numbers = [Decimal(x) for x in args]
    total = sum(numbers)
    return total / len(numbers)
